In [1]:
from opentree import OT
import physcraper ## NEEDS branch ebird
import os

In [2]:
infi = open("../taxonomy_info/combined_data.tsv")
header = infi.readline().split('\t')

double_match = {}
mapping_dict = {}

for lin in infi:
    lii = lin.split('\t')
    if lii[9] == '-':
        continue
    if lii[9] in mapping_dict:
        if lii[9] not in double_match:
            double_match[lii[9]] = {}
        existing_eb_nam = mapping_dict[lii[9]]['SCI_NAME']
        double_match[lii[9]][existing_eb_nam] = mapping_dict[lii[9]]
        new_eb_name = lii[4]
        double_match[lii[9]][new_eb_name] = dict(zip(header, lii))
    mapping_dict[lii[9]]=dict(zip(header, lii))
    

infi.close()

In [3]:
aves_trees_resp = OT.find_trees("81461", search_property ="ot:ottId")

In [4]:
all_aves_resp = OT.taxon_subtree(ott_id = "81461", label_format = "id")

In [5]:
all_aves = set(all_aves_resp.response_dict['newick'].replace('(', ',').replace(')',',').replace('ott', ',').split(','))
all_aves.remove('')

In [6]:
forwards = open("../../ott3.2/forwards.tsv")
forward_dict = {}
for lin in forwards:
    lii = lin.strip().split('\t')
    forward_dict[lii[0]] = lii[1]
    if lii[1].strip() in all_aves:
        all_aves.add(lii[0])

In [7]:
len(all_aves)

29249

In [8]:
len(aves_trees_resp.response_dict['matched_studies'])

342

In [14]:
tips_mapped_with_no_ebird_match = {}
study_to_cite = {}
#os.mkdir("../bird_studies_translation")
#os.chdir("../bird_studies_translation")


In [15]:
for study in aves_trees_resp.response_dict['matched_studies']:
    study_id = study['ot:studyId']
    curr_dir = "study_{}".format(study_id)
    try:
        os.mkdir(curr_dir)
    except:
        pass
    cite = OT.get_citations([study_id])
    study_to_cite[study_id] = cite
    fi = open("{}/cite.txt".format(curr_dir), 'w')
    fi.write(cite)
    fi.close()
    for tree in study['matched_trees']:
        tree_id = tree['ot:treeId']
        tt = physcraper.opentree_helpers.generate_treetaxon_from_phylesystem(study_id=study_id, tree_id=tree_id)
        for tip in tt.otu_dict:
            ott_id = str(tt.otu_dict[tip]['^ot:ottId'])
            if ott_id in forward_dict:
                ott_id = forward_dict[ott_id]
            if ott_id not in all_aves:
                spp_code = "not_a_bird"
                ebird_name = "not_a_bird"
            else:
                ebird_info = mapping_dict.get(ott_id, None)
                if ebird_info:
                    spp_code = ebird_info['SPECIES_CODE']
                    ebird_name = ebird_info['SCI_NAME']
                else:
                    spp_code = "no_match"
                    ebird_name = "no_match"
                    if ott_id in tips_mapped_with_no_ebird_match:
                        tips_mapped_with_no_ebird_match[ott_id].append("{},{}".format(tt.otu_dict[tip]['^ot:originalLabel'],study_id))
                    else:
                        tips_mapped_with_no_ebird_match[ott_id] = ["{},{}".format(tt.otu_dict[tip]['^ot:originalLabel'],study_id)]
            tt.otu_dict[tip]['^clo:speciescode2019'] = spp_code
            tt.otu_dict[tip]['^clo:ClementsTaxonName2019'] = ebird_name
        physcraper.aligntreetax.write_otu_file(tt, "{}/{}_{}_labels.csv".format(curr_dir, study_id, tree_id), 'table', headers = "other")
        for tip in tt.tre.leaf_node_iter():
            tip.annotations['otu'] = tip.label
            for prop in ['^ot:ottId','^ot:ottTaxonName','^ot:originalLabel', '^clo:ClementsTaxonName2019', '^clo:speciescode2019']:
                tip.annotations[prop] =  tt.otu_dict[tip.label][prop]
        tt.tre.annotations['messages'] = "" #it won't write out if there are json annotations in the messages
        tt.tre.write(path="{}/{}_{}.nex".format(curr_dir, study_id, tree_id), schema="nexus")


In [17]:
ott_fi = open("tips_with_no_Clements_match.tsv", 'w')
ott_fi.write('ott_id\toriginal_label,study_id')
for ott_id in tips_mapped_with_no_ebird_match:
    ott_fi.write("{}\t".format(ott_id))
    ott_fi.write("\t".join(tips_mapped_with_no_ebird_match[ott_id]))
    ott_fi.write("\n")
ott_fi.close()
    
    

In [18]:
cite_fi = open("citation_map.tsv", 'w')
for study in study_to_cite:
    cite = study_to_cite[study].replace('\n','\t')
    cite_fi.write("{}\t{}\n".format(study, cite))
    
cite_fi.close()
